# Thông tin sinh viên
* Họ tên: Mai Duy Nam
* MSSV: 19120298

# Đọc dữ liệu

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('weatherPrediction2.csv')
df.head()

,Outlook,Temperature,Humidity,Wind speed,Play cricket
0,Sunny,NaN,High,4.0,No
1,Sunny,NaN,High,20.0,No
2,Overcast,Hot,High,6.0,Yes
3,Rain,Mild,High,2.0,Yes
4,Rain,Cool,Normal,7.0,Yes


# Điền dữ liệu thiếu

Các cột có chứa giá trị thiếu:

In [3]:
missing_cols = df.columns[df.isna().any()]
missing_cols

Index(['Temperature', 'Wind speed'], dtype='object')

Có hai cột có chứa giá trị thiếu là Temperature và Wind speed. Ta nhận thấy hai cột này chứa dạng dữ liệu khác nhau: cột Temperature có dữ liệu dạng phân loại, còn cột Wind speed có dữ liệu dạng số. Do đó, ta sẽ điền giá trị thiếu cho hai cột này với các chiến lược khác nhau.

## Một số hàm và biến phụ trợ

In [4]:
original_missing_cells = dict()
for col in missing_cols:
    x = df[col]
    original_missing_cells[col] = list(x[x.isna()].to_dict().keys())
original_missing_cells

{'Temperature': [0, 1, 5, 7, 12], 'Wind speed': [5, 10]}

In [5]:
def slice_dict(d, key):
    return {key: d[key]} if key in d else None

In [6]:
def highlight_originally_missing_cells(df, to_highlight):
    def highlighter(x, to_highlight):
        if x.name not in to_highlight.keys():
            return [""] * len(x)
        return ['color: red' if v in to_highlight[x.name] else "" for v in x.index]
    return df.style.apply(highlighter, axis=0, to_highlight=to_highlight)

## Cột Wind speed

In [7]:
ws_missing_cells = slice_dict(original_missing_cells, key='Wind speed')
ws_missing_cells

{'Wind speed': [5, 10]}

### Cách 1: Điền bằng giá trị trung bình

In [8]:
ws_filled_mean_df = df.fillna({'Wind speed': df['Wind speed'].mean()})
highlight_originally_missing_cells(ws_filled_mean_df, ws_missing_cells)

,Outlook,Temperature,Humidity,Wind speed,Play cricket
0,Sunny,nan,High,4.000000,No
1,Sunny,nan,High,20.000000,No
2,Overcast,Hot,High,6.000000,Yes
3,Rain,Mild,High,2.000000,Yes
4,Rain,Cool,Normal,7.000000,Yes
5,Rain,nan,Normal,8.666667,No
6,Overcast,Cool,Normal,22.000000,Yes
7,Sunny,nan,High,4.000000,No
8,Sunny,Cool,Normal,5.000000,Yes
9,Rain,Mild,Normal,6.000000,Yes


### Cách 2: Điền bằng giá trị trung vị

In [9]:
ws_filled_median_df = df.fillna({'Wind speed': df['Wind speed'].median()})
highlight_originally_missing_cells(ws_filled_median_df, ws_missing_cells)

,Outlook,Temperature,Humidity,Wind speed,Play cricket
0,Sunny,nan,High,4.000000,No
1,Sunny,nan,High,20.000000,No
2,Overcast,Hot,High,6.000000,Yes
3,Rain,Mild,High,2.000000,Yes
4,Rain,Cool,Normal,7.000000,Yes
5,Rain,nan,Normal,6.000000,No
6,Overcast,Cool,Normal,22.000000,Yes
7,Sunny,nan,High,4.000000,No
8,Sunny,Cool,Normal,5.000000,Yes
9,Rain,Mild,Normal,6.000000,Yes


### Cách 3: Điền bằng giá trị mode

In [10]:
df['Wind speed'].mode()

0    2.0
1    4.0
2    6.0
Name: Wind speed, dtype: float64

Vì cột Wind speed có nhiều giá trị mode, ta mặc định chọn giá trị mode đầu tiên.

In [11]:
ws_filled_mode_df = df.fillna({'Wind speed': df['Wind speed'].mode()[0]})
highlight_originally_missing_cells(ws_filled_mode_df, ws_missing_cells)

,Outlook,Temperature,Humidity,Wind speed,Play cricket
0,Sunny,nan,High,4.000000,No
1,Sunny,nan,High,20.000000,No
2,Overcast,Hot,High,6.000000,Yes
3,Rain,Mild,High,2.000000,Yes
4,Rain,Cool,Normal,7.000000,Yes
5,Rain,nan,Normal,2.000000,No
6,Overcast,Cool,Normal,22.000000,Yes
7,Sunny,nan,High,4.000000,No
8,Sunny,Cool,Normal,5.000000,Yes
9,Rain,Mild,Normal,6.000000,Yes


### Cách 4: Điền bằng giá trị trung bình/trung vị nhóm theo cột Outlook

Ta điền giá trị thiếu của cột Wind speed dựa theo giá trị trung bình/trung vị theo từng nhóm của cột Outlook: Overcast, Rain hay Sunny. Các cell dưới đây tính giá trị trung bình, trung vị và mode của các cột, nhóm theo cột Outlook.

In [12]:
group_by_outlook_mean = df.groupby('Outlook').mean()
group_by_outlook_mean

,Wind speed
Outlook,
Overcast,9.50
Rain,8.25
Sunny,8.25


In [13]:
group_by_outlook_median = df.groupby('Outlook').median()
group_by_outlook_median

,Wind speed
Outlook,
Overcast,7.0
Rain,6.5
Sunny,4.5


In [14]:
group_by_outlook_mode = df.groupby('Outlook').agg(lambda x: x.mode()[0])
group_by_outlook_mode

,Temperature,Humidity,Wind speed,Play cricket
Outlook,,,,
Overcast,Cool,High,2.0,Yes
Rain,Mild,Normal,2.0,Yes
Sunny,Cool,High,4.0,No


Các cell dưới đây chuyển bảng giá trị trung bình, trung vị và mode của cột Wind speed thành dạng `dict`.

In [15]:
replace_by_mean = group_by_outlook_mean.to_dict()['Wind speed']
replace_by_mean

{'Overcast': 9.5, 'Rain': 8.25, 'Sunny': 8.25}

In [16]:
replace_by_median = group_by_outlook_median.to_dict()['Wind speed']
replace_by_median

{'Overcast': 7.0, 'Rain': 6.5, 'Sunny': 4.5}

In [17]:
replace_by_mode = group_by_outlook_mode['Wind speed'].to_dict()
replace_by_mode

{'Overcast': 2.0, 'Rain': 2.0, 'Sunny': 4.0}

Ta sẽ thay giá trị thiếu trong Wind speed bằng cách duyệt từng dòng của `df` (sử dụng phương thức `apply` của `DataFrame`). Hàm `replace_nan` dưới đây thay thế giá trị thiếu trong cột `replace_col` của một dòng `x` bằng giá trị trung bình/trung vị tương ứng với từng nhóm trong Outlook. Tham số `replace_dict` lưu các giá trị thay thế.

In [18]:
def replace_nan(x, replace_dict, replace_col):
    if pd.isna(x[replace_col]):
        x[replace_col] = replace_dict[x['Outlook']]
    return x

Ta thay thế bằng cách `apply` hàm `replace_nan` theo từng dòng.

In [19]:
ws_replaced_mean_df = df.apply(replace_nan, axis=1, replace_dict=replace_by_mean, replace_col='Wind speed')
highlight_originally_missing_cells(ws_replaced_mean_df, ws_missing_cells)

,Outlook,Temperature,Humidity,Wind speed,Play cricket
0,Sunny,nan,High,4.000000,No
1,Sunny,nan,High,20.000000,No
2,Overcast,Hot,High,6.000000,Yes
3,Rain,Mild,High,2.000000,Yes
4,Rain,Cool,Normal,7.000000,Yes
5,Rain,nan,Normal,8.250000,No
6,Overcast,Cool,Normal,22.000000,Yes
7,Sunny,nan,High,4.000000,No
8,Sunny,Cool,Normal,5.000000,Yes
9,Rain,Mild,Normal,6.000000,Yes


In [20]:
ws_replaced_median_df = df.apply(replace_nan, axis=1, replace_dict=replace_by_median, replace_col='Wind speed')
highlight_originally_missing_cells(ws_replaced_median_df, ws_missing_cells)

,Outlook,Temperature,Humidity,Wind speed,Play cricket
0,Sunny,nan,High,4.000000,No
1,Sunny,nan,High,20.000000,No
2,Overcast,Hot,High,6.000000,Yes
3,Rain,Mild,High,2.000000,Yes
4,Rain,Cool,Normal,7.000000,Yes
5,Rain,nan,Normal,6.500000,No
6,Overcast,Cool,Normal,22.000000,Yes
7,Sunny,nan,High,4.000000,No
8,Sunny,Cool,Normal,5.000000,Yes
9,Rain,Mild,Normal,6.000000,Yes


In [21]:
ws_replaced_mode_df = df.apply(replace_nan, axis=1, replace_dict=replace_by_mode, replace_col='Wind speed')
highlight_originally_missing_cells(ws_replaced_mode_df, ws_missing_cells)

,Outlook,Temperature,Humidity,Wind speed,Play cricket
0,Sunny,nan,High,4.000000,No
1,Sunny,nan,High,20.000000,No
2,Overcast,Hot,High,6.000000,Yes
3,Rain,Mild,High,2.000000,Yes
4,Rain,Cool,Normal,7.000000,Yes
5,Rain,nan,Normal,2.000000,No
6,Overcast,Cool,Normal,22.000000,Yes
7,Sunny,nan,High,4.000000,No
8,Sunny,Cool,Normal,5.000000,Yes
9,Rain,Mild,Normal,6.000000,Yes


## Cột Temperature

In [22]:
temp_missing_cells = slice_dict(original_missing_cells, key='Temperature')
temp_missing_cells

{'Temperature': [0, 1, 5, 7, 12]}

### Cách 1: Điền bằng giá trị mode

In [23]:
df['Temperature'].mode()

0    Mild
Name: Temperature, dtype: object

In [24]:
temp_filled_mode_df = df.fillna({'Temperature': df['Temperature'].mode()[0]})
highlight_originally_missing_cells(temp_filled_mode_df, temp_missing_cells)

,Outlook,Temperature,Humidity,Wind speed,Play cricket
0,Sunny,Mild,High,4.000000,No
1,Sunny,Mild,High,20.000000,No
2,Overcast,Hot,High,6.000000,Yes
3,Rain,Mild,High,2.000000,Yes
4,Rain,Cool,Normal,7.000000,Yes
5,Rain,Mild,Normal,nan,No
6,Overcast,Cool,Normal,22.000000,Yes
7,Sunny,Mild,High,4.000000,No
8,Sunny,Cool,Normal,5.000000,Yes
9,Rain,Mild,Normal,6.000000,Yes


### Cách 2: Điền bằng giá trị mode, nhóm theo cột Outlook

Tương tự như cách ta điền giá trị thiếu trong cột Wind speed bằng cách nhóm theo Outlook, ở đây ta áp dụng tương tự với cột Temperature, sử dụng thông số là mode. Ta sử dụng lại biến `group_by_outlook_mode` và hàm `replace_nan`.

In [25]:
replace_temperature_by_mode = group_by_outlook_mode['Temperature'].to_dict()
replace_temperature_by_mode

{'Overcast': 'Cool', 'Rain': 'Mild', 'Sunny': 'Cool'}

In [26]:
temp_replaced_mode_df = df.apply(replace_nan, axis=1, replace_dict=replace_temperature_by_mode, replace_col='Temperature')
highlight_originally_missing_cells(temp_replaced_mode_df, temp_missing_cells)

,Outlook,Temperature,Humidity,Wind speed,Play cricket
0,Sunny,Cool,High,4.000000,No
1,Sunny,Cool,High,20.000000,No
2,Overcast,Hot,High,6.000000,Yes
3,Rain,Mild,High,2.000000,Yes
4,Rain,Cool,Normal,7.000000,Yes
5,Rain,Mild,Normal,nan,No
6,Overcast,Cool,Normal,22.000000,Yes
7,Sunny,Cool,High,4.000000,No
8,Sunny,Cool,Normal,5.000000,Yes
9,Rain,Mild,Normal,6.000000,Yes


## Điền dữ liệu thiếu cho cả dataframe

Để điền dữ liệu thiếu cho cả dataframe, ta có thể điền mỗi cột một cách độc lập. Tổng cộng ta có 4 $\times$ 2 = 8 cách điền.